In [20]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [21]:
# Create imbalanced binary classification dataset
X, y = make_classification(n_samples=1000,
                           n_features=10,
                           n_informative=2,
                           n_redundant=8,
                           weights=[0.9, 0.1],
                           flip_y=0,
                           random_state=42
                           )

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [22]:
# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    stratify=y,
                                                    random_state=42)

In [23]:
# Define model hyperparameters

params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 42
}

lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [24]:
report_dict = classification_report(y_test, y_pred, output_dict=True)

In [25]:
import mlflow

In [26]:
mlflow.set_experiment("1st experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [27]:
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0': report_dict['0']['recall'],
        'recall_class_1': report_dict['1']['recall'],
        'f1_score_macro': report_dict['macro avg']['f1-score'],
    })
    mlflow.sklearn.log_model(lr, "LogisticRegression")

2025/02/28 11:50:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run trusting-flea-856 at: http://127.0.0.1:5000/#/experiments/405825615780292970/runs/4b8b1a9242cc4f678a1c9210d80bc5e3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/405825615780292970


## Experiment 2: Random Forest Classifier

In [28]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3, random_state=42)
rf_clf.fit(X_train, y_train)

y_pred_clf = rf_clf.predict(X_test)

report_clf = classification_report(y_test, y_pred_clf)
print(report_clf)

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



## Experiment 3: XGBoost

In [ ]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)

report_xgb = classification_report(y_test, y_pred_xgb)
print(report_xgb)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



## Experiment 4: SMOTE and XGBoost

In [30]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [31]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
report_xgb = classification_report(y_test, y_pred_xgb)
print(report_xgb)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



In [35]:
"""
Track experiments with MLflow
"""
models = [
    (
        "LogisticRegression",
        LogisticRegression(C=1, solver="liblinear"),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "RandomForest",
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier_SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [ ]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [ ]:
mlflow.set_experiment("Anomaly Detection")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model_name", model_name)
        mlflow.log_params(model.get_params())
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_0': report['0']['recall'],
            'recall_class_1': report['1']['recall'],
            'f1_score_macro': report['macro avg']['f1-score'],
        })

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, model_name)
        else:
            mlflow.sklearn.log_model(model, model_name)

2025/02/28 12:05:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression at: http://127.0.0.1:5000/#/experiments/310292253855914995/runs/cef6942550334df784f39a75e516dc20
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/310292253855914995


2025/02/28 12:05:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForest at: http://127.0.0.1:5000/#/experiments/310292253855914995/runs/f022b0b797284863b27098449daee095
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/310292253855914995


2025/02/28 12:05:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/310292253855914995/runs/6f48aca7828f4c3092952c3266128e26
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/310292253855914995


2025/02/28 12:05:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier_SMOTE at: http://127.0.0.1:5000/#/experiments/310292253855914995/runs/d9466a3b384f48b981582e0e37c76c03
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/310292253855914995
